In [1]:
import numpy as np
import pandas as pd
import scipy.stats as st
from scipy.stats import chi2_contingency, chi2 
import seaborn as sns
import matplotlib.pyplot as plt
import pingouin as pg

In [2]:
logs = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-rom-fadeev/KC_case_data .csv')
logs.head(5)

/opt/tljh/user/lib/python3.7/site-packages/outdated/utils.py:18: OutdatedCacheFailedWarning: Failed to use cache while checking for outdated package.
Set the environment variable OUTDATED_RAISE_EXCEPTION=1 for a full traceback.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs
/opt/tljh/user/lib/python3.7/site-packages/outdated/utils.py:18: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.1, the latest is 0.2.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs
/opt/tljh/user/lib/python3.7/site-packages/outdated/utils.py:18: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.4.0, the latest is 0.5.5.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs


,date,event,purchase_sum,os_name,device_id,gender,city,utm_source
0,2020-01-01,app_start,NaN,android,669460,female,Moscow,-
1,2020-01-01,app_start,NaN,ios,833621,male,Moscow,vk_ads
2,2020-01-01,app_start,NaN,android,1579237,male,Saint-Petersburg,referal
3,2020-01-01,app_start,NaN,android,1737182,female,Moscow,facebook_ads
4,2020-01-01,app_start,NaN,ios,4029024,female,Moscow,facebook_ads


In [22]:
# ___№11_________пришедшие с какого канала, имеют медианный первый чек выше? (учитываются только первые покупки)

df_med = logs

df_med = df_med.query("event == 'purchase'" )\
                .drop_duplicates(subset='device_id', keep='first')\
                .groupby(["utm_source"], as_index=False)\
                .agg(device_id=('device_id' , 'unique'), mediana=('purchase_sum' , 'median'))


df_med = df_med.sort_values('mediana', ascending=False)
df_med

,utm_source,device_id,mediana
0,-,"[6081155, 16780017, 11913582, 52165, 11734309,...",398.5
4,referal,"[26467125, 16413807, 8152843, 10055437, 111017...",395.5
3,instagram_ads,"[7458607, 3520947, 20207733, 43390, 24508327, ...",393.5
5,vk_ads,"[740467, 146534, 503757, 11624895, 8165320, 10...",393.0
6,yandex-direct,"[77448, 27742079, 9545205, 32829588, 10345186,...",392.5
2,google_ads,"[1170605, 27612889, 8336897, 17385305, 3774946...",390.5
1,facebook_ads,"[166319, 7351500, 33478729, 30781321, 25727658...",389.0


In [4]:
# _____________№12____________________Какой платный канал привлечения (среди рекламных) имеет самый высокий ROMI?
# ROMI — измерить, как расходы на маркетинг влияют на доходы.


df_zatr = pd.DataFrame([['facebook_ads',8590498],
                  ['google_ads',10534878],
                  ['instagram_ads',8561626],
                  ['vk_ads',9553531],
                  ['yandex-direct',10491707]],
columns=['utm_source', 'zatrat'])


df_romi = logs
df_romi = df_romi.groupby(["utm_source"], as_index=False)\
                .agg(revenue=('purchase_sum' , 'sum'))

df_romi = df_romi.merge(df_zatr, how='left', on='utm_source')
df_romi['ROMI'] = ((df_romi['revenue'] - df_romi['zatrat'])/df_romi['zatrat'])*100
df_romi.sort_values('ROMI', ascending=False)

,utm_source,revenue,zatrat,ROMI
5,vk_ads,16389652.5,9553531.0,71.555967
3,instagram_ads,14546969.0,8561626.0,69.908952
1,facebook_ads,12249901.0,8590498.0,42.598264
6,yandex-direct,13915368.0,10491707.0,32.632068
2,google_ads,12868276.0,10534878.0,22.149265
0,-,21449749.5,NaN,NaN
4,referal,8837044.5,NaN,NaN


In [5]:
# ___№10____________CR = кол-во юзеров, впервые совершивших покупку/кол-во юзеров, впервые открывших приложение 


df_chanels = logs
df_chanels_appSt = df_chanels.query("event == 'app_start'" )\
                    .groupby(["utm_source"], as_index=False)\
                    .agg(app_start_count=('device_id' , 'nunique'))

df_chanels_purch = df_chanels.query("event == 'purchase'" )\
                    .groupby(["utm_source"], as_index=False)\
                    .agg(purchase_count=('device_id' , 'nunique'))

df_chanels_comm = df_chanels_appSt.merge(df_chanels_purch, how='left', on='utm_source')
df_chanels_comm['CR'] = df_chanels_comm['purchase_count']/df_chanels_comm['app_start_count']
df_chanels_comm = df_chanels_comm.sort_values('CR', ascending=False)
df_chanels_comm

,utm_source,app_start_count,purchase_count,CR
4,referal,15924,6362,0.399523
5,vk_ads,34079,12364,0.362804
1,facebook_ads,25957,9017,0.347382
3,instagram_ads,31045,10762,0.346658
0,-,52268,16598,0.317556
2,google_ads,38092,11339,0.297674
6,yandex-direct,40707,12028,0.295477


In [6]:
#_________________№9___________________________________________ кол-во отвалившихся пользователей в воронке продаж

In [7]:
df_all = logs
df_reg = df_all.query("event == 'register'" )\
                    .groupby(["device_id","date"], as_index=False)\
                    .agg(date_purchase=('gender' , 'count'))

# gg.groupby(["event"], as_index=False).count().sort_values('date', ascending=False)

common = df_all.merge(df_reg, how='left', on='device_id')
common = common.rename(columns={'date_y': "date_reg", 'date_x': "date"})
del common['purchase_sum']
df_common = common.drop(['os_name', 'gender', 'utm_source', 'date_purchase', 'city'], axis=1)
df_common['with_reg'] = df_common['date_reg'] < df_common['date']
# .query("device_id ==669460" )
df_common_reg = df_common.query("with_reg ==True" )\
                    .groupby(["event"], as_index=False)\
                    .agg(count=('event' , 'count'))\
                    .sort_values('count', ascending=False)
df_common_reg = df_common_reg.rename(columns={'count': "reg"})

df_common_not_reg = df_common.query("with_reg == False" )\
                    .groupby(["event"], as_index=False)\
                    .agg(count=('event' , 'count'))\
                    .sort_values('count', ascending=False)
df_common_not_reg = df_common_not_reg.rename(columns={'count': "not_reg"})

df_common_reg_notreg = df_common_reg.merge(df_common_not_reg, how='inner', on='event')
df_common_reg_notreg

,event,reg,not_reg
0,app_start,441486,307219
1,search,417101,291538
2,choose_item,314473,224196
3,tap_basket,217256,160409
4,purchase,73630,67753


In [8]:
#_________№7___________________________________________________ конверсия в течении 7 дней по когортам

In [9]:
# logs = logs.query("device_id ==4014 and event == 'app_start'" )
#             .groupby(["utm_source"], as_index=False)\
#             .agg({'event' : 'count'})\
#             .sort_values('event', ascending=False)
# logs

In [10]:
logs['date'] = pd.to_datetime(logs['date'])

# inst['device_id'].nunique() == 154597 и count() такой же
inst = logs.query("event == 'app_install'" )\
    .groupby(["device_id","date"], as_index=False)\
    .agg({'event' : 'count'})
inst = inst.rename(columns={'event': "install",'date': "date_install" })

# purch['device_id'].nunique() == 70875   purch['device_id'].count() == 141383
# тк покупок у одного id может быть несколько в разные даты , убираем дубликаты
purch = logs.query("event == 'purchase'" )\
    .groupby(["device_id","date"], as_index=False)\
    .agg({'event' : 'count'}) 
purch = purch.rename(columns={'event': "purchase",'date': "date_purchase"})
# тк главное- это первая покупка, то удаляем дубликаты покупок по id
purch = purch.drop_duplicates(subset = ['device_id'])

all_ev = inst.merge(purch, how='left', on='device_id')
#  7 дней с даты установки
all_ev['day'] = (all_ev['date_purchase'] - all_ev['date_install']).dt.days
all_ev['day'] = all_ev['day'].fillna(0)
all_ev['delta']=all_ev['date_install'] + pd.Timedelta(days=7)

# только покупки             
purch1 = all_ev.query("day<=7.0 and purchase != 'NaN' ")\
    .groupby(['date_install'], as_index=False)\
    .agg({'device_id' : 'nunique'})
# все установки без  day<=7.0 тк неважно купили или нет, нужно общее кол-во установок
inst1 = all_ev.groupby(['date_install'], as_index=False)\
    .agg({'install' : 'count'})

com = purch1.merge(inst1, how='left', on='date_install') 
com = com.rename(columns={'device_id': "purchase"})
com['CR'] = ((com['purchase']/com['install'])*100).round(1) 
com = com.sort_values('CR', ascending=False)
com

,date_install,purchase,install,CR
0,2020-01-01,1408,3579,39.3
8,2020-01-09,558,1424,39.2
14,2020-01-15,1650,4310,38.3
13,2020-01-14,1973,5173,38.1
1,2020-01-02,1186,3144,37.7
...,...,...,...,...
77,2020-03-18,235,1171,20.1
88,2020-03-29,223,1117,20.0
81,2020-03-22,251,1261,19.9
87,2020-03-28,209,1091,19.2


In [ ]:
# _______________________________  преобразует в дни
# .days

In [ ]:
#  взять первое значение         .drop_duplicates(subset='device_id', keep='first')
# 
# df_med = df_med.query("event == 'purchase'" )\
#                 .drop_duplicates(subset='device_id', keep='first')\  <------------------------------------!!!!!!!!!!!
#                 .groupby(["utm_source"], as_index=False)\
#                 .agg(device_id=('device_id' , 'unique'), mediana=('purchase_sum' , 'median'))


In [11]:
# .agg(date_purchase=('gender' , 'count'))

# del common['purchase_sum']  - удаление столбца
# df_common = common.drop(['os_name', 'gender', 'utm_source', 'date_purchase', 'city'], axis=1)

# .value_counts()   -  подсчитать количество значений в конкретном столбце

In [12]:
# inst['device_id'].nunique()

In [13]:
# df_common.tail(50)
# df_common.head(50)

In [14]:
# inst.drop_duplicates(subset = ['device_id']).count()
# inst = logs.query("event == 'app_install'" ).drop_duplicates(subset = ['device_id'])
# inst['device_id'].count()

In [15]:
# logs.drop_duplicates(subset = ['device_id']).count()

In [16]:
# all_ev['cogorts']=all_ev['date_install'] + pd.Timedelta(days=7)


In [17]:
# all_ev['ggggg'] = all_ev['cogort'] <= all_ev['date_install'] <= all_ev['delta']

In [18]:
# logs['date'] = pd.to_datetime(logs['date'])

# # inst['device_id'].nunique() == 154597 и count() такой же
# inst = logs.query("event == 'app_install'" )\
#     .groupby(["device_id","date"], as_index=False)\
#     .agg({'event' : 'count'})
# inst = inst.rename(columns={'event': "install",'date': "date_install" })

# # purch['device_id'].nunique() == 70875   purch['device_id'].count() == 141383
# purch = logs.query("event == 'purchase'" )\
#     .groupby(["device_id","date"], as_index=False)\
#     .agg({'event' : 'count'}) 
# purch = purch.rename(columns={'event': "purchase",'date': "date_purchase"})

# all_ev = inst.merge(purch, how='left', on='device_id') 
# all_ev['day'] = (all_ev['date_purchase'] - all_ev['date_install']).dt.days
# all_ev['day'] = all_ev['day'].fillna(0)
# all_ev['cogort'] = all_ev['date_install'] + pd.Timedelta(days=7)
# all_ev['delta'] = pd.Timedelta(all_ev['date_install'], all_ev['cogort'])

# purch = all_ev.query("day<=7.0 and purchase != 'NaN'")\
#     .groupby(['date_install'], as_index=False)\
#     .agg({'device_id' : 'nunique'})

# inst = all_ev.groupby(['date_install'], as_index=False)\
#     .agg({'install' : 'count'})

# com = purch.merge(inst, how='inner', on='date_install') 
# # com = com.rename(columns={'device_id': "purchase"})
# # com['CR'] = ((com['purchase']/com['install'])*100).round(1) 
# # com = com.sort_values('CR', ascending=False)
# all_ev